In [1]:
import pandas as pd
import json
import sklearn.svm as svm
import numpy as np
from sklearn.utils import shuffle

In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [3]:
import script as sc

In [8]:
def load_Data(user, filename):
   data = json.load(open('../dir.json'))
   return (data[user] + filename)

In [3]:
def load_struct():
    data = json.load(open('../struct.json'))
    return data

In [11]:
filelist = load_struct()
observedData = pd.DataFrame()
user = "AndrewStaudenmeyer"

In [13]:
files = filelist["1"]

In [16]:
data = pd.read_csv(load_Data(user, files["AG"]), header=0)
gt1 = pd.read_csv(load_Data(user, files["GT1"]), header=0)
type1 = files["GT1"][2]
start1 = files["StartTime1"]
end1 = files["EndTime1"]

In [17]:
data.head()

,Unnamed: 0,mean.vm,sd.vm,mean.ang,sd.ang,p625,dfreq,ratio.df,sed.rf,start.time
0,1,1.008720,0.005388,14.405906,0.178485,0.073094,6.153846,0.079059,sedentary,2017-10-02 10:15:00
1,2,1.008625,0.006601,14.444647,0.234531,0.074642,4.102564,0.174183,sedentary,2017-10-02 10:15:01
2,3,1.018935,0.023567,12.862005,2.996361,0.309095,1.025641,0.158892,sedentary,2017-10-02 10:15:02
3,4,1.019761,0.066683,-1.952166,12.679718,0.114479,2.051282,0.101853,sedentary,2017-10-02 10:15:03
4,5,1.043682,0.056193,-16.001678,5.775647,0.217691,1.025641,0.149609,sedentary,2017-10-02 10:15:04


In [18]:
observedData = pd.DataFrame(useable_Data(data, gt1, start1, end1))

observedData['type'] = type1 + '-' + str(subject)

if("GT2" in files):
    gt2 = pd.read_csv(load_Data(user, files["GT2"]), header=0)
    type2 = files["GT2"][2]
    start2 = files["StartTime2"]
    end2 = files["EndTime2"]
    observedData1 = pd.DataFrame(useable_Data(data, gt2, start2, end2))
    observedData1['type'] = type2 + '-' + str(subject)
    observedData = pd.concat([observedData, observedData1])

NameError: name 'useable_Data' is not defined

In [ ]:
observedData1 = get_observed_data_for_subject(user, i, files[str(1)])
observedData = pd.concat([observedData, observedData1]) 

In [4]:
sc.update_staudenmeyer_complete("AndrewStaudenmeyer")

C:\Users\Andrew\Documents\CalPoly\DataCapstone\KineseItUp\script.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  observedData['index'] = range(len(observedData))


In [5]:
staudenmeyer_data = sc.get_staudenmeyer_complete("AndrewStaudenmeyer")

In [6]:
staudenmeyer_data.head()

,Unnamed: 0,Unnamed: 0.1,mean.vm,sd.vm,mean.ang,sd.ang,p625,dfreq,ratio.df,sed.rf,start.time,index,activity,coding,type
0,0,368947,1.027036,0.065807,-32.179828,8.090852,0.188766,4.102564,0.126940,non-sedentary,2017-10-06 16:44:06,0,active,non-sedentary,H-1
1,1,368948,1.029727,0.136367,-32.955579,8.908046,0.222211,3.076923,0.174922,non-sedentary,2017-10-06 16:44:07,1,active,non-sedentary,H-1
2,2,368949,1.003435,0.056664,-47.568056,6.244429,0.235925,3.076923,0.245706,non-sedentary,2017-10-06 16:44:08,2,active,non-sedentary,H-1
3,3,368950,1.020198,0.070674,-54.486552,4.062847,0.479812,2.051282,0.241239,non-sedentary,2017-10-06 16:44:09,3,active,non-sedentary,H-1
4,4,368951,1.003209,0.031647,-52.630436,2.084940,0.443848,2.051282,0.291673,sedentary,2017-10-06 16:44:10,4,active,non-sedentary,H-1


In [7]:
correct = staudenmeyer_data['sed.rf'] == staudenmeyer_data['coding']

In [8]:
correct[correct] = 1

In [9]:
sum(correct) / len(correct)

0.696641116719636

In [14]:
staudenmeyer_data.loc[staudenmeyer_data['sed.rf'] != 'sedentary', 'sed.rf'] = 0
staudenmeyer_data.loc[staudenmeyer_data['sed.rf'] == 'sedentary', 'sed.rf'] = 1

In [15]:
staudenmeyer_data.groupby(staudenmeyer_data['type'])['sed.rf'].sum()

type
A-13    2699
A-4     1996
A-6     2604
A-8     2752
E-14    4465
E-6     4443
E-7     4543
E-8     4450
H-1     4009
H-3     4903
H-7     4061
L-1     6258
L-3     6293
L-5     6186
W-13    6103
W-15    5862
W-4     3106
W-9     4842
Name: sed.rf, dtype: int64